In [ ]:
!pip install pygooglenews --upgrade
!pip install pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 7.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=1c9a1e258ab39215ccc54e16b0f6ad41f08f48f4851c592881a3fe8a9e1efd87
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [ ]:
#@title Indicateur de Volume Relatif (Volume)
#@markdown Saisissez les tickers (ex: AAPL, TSLA, NVDA, XOM) :
liste_tickers = "XOM, NVDA, CVX, AAPL, BNP.PA" #@param {type:"string"}

!pip install yfinance --quiet

import yfinance as yf
import pandas as pd
import numpy as np

def calculate_volume_indicator(ticker):
    try:
        # 1. Récupération de l'historique sur 1 an (252 jours de bourse)
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1y")

        if len(hist) < 30:
            return None

        # 2. Données de volume
        current_volume = hist['Volume'].iloc[-1]
        volume_hist = hist['Volume']

        avg_vol = volume_hist.mean()
        max_vol = volume_hist.max()
        min_vol = volume_hist.min()

        # 3. Calcul de l'indicateur entre -1 et 1
        # Centré sur la moyenne. 1 = Max annuel, -1 = Min annuel.
        if (max_vol - min_vol) != 0:
            indicator = (current_volume - avg_vol) / (max_vol - min_vol)
        else:
            indicator = 0

        return {
            'Ticker': ticker,
            'Volume Jour': f"{current_volume:,.0f}",
            'Moyenne Annuelle': f"{round(avg_vol):,.0f}",
            'Indicateur (-1 à 1)': round(np.clip(indicator, -1, 1), 3)
        }
    except Exception as e:
        print(f"Erreur pour {ticker}: {e}")
        return None

# --- EXÉCUTION ---
tickers = [t.strip().upper() for t in liste_tickers.split(',')]
results = []

print(f"🔍 Analyse du volume relatif pour {len(tickers)} tickers...")

for ticker in tickers:
    res = calculate_volume_indicator(ticker)
    if res:
        results.append(res)
        print(f"✅ {ticker} terminé.")

# --- AFFICHAGE ---
if results:
    df_final = pd.DataFrame(results)
    print("\n" + "="*70)
    print(" TABLEAU DE BORD : ANOMALIES DE VOLUME (HYPER-FIABLE)")
    print("="*70)
    display(df_final.sort_values(by='Indicateur (-1 à 1)', ascending=False))
else:
    print("Aucune donnée n'a pu être récupérée.")

🔍 Analyse du volume relatif pour 5 tickers...
✅ XOM terminé.
✅ NVDA terminé.
✅ CVX terminé.
✅ AAPL terminé.
✅ BNP.PA terminé.

 TABLEAU DE BORD : ANOMALIES DE VOLUME (HYPER-FIABLE)


,Ticker,Volume Jour,Moyenne Annuelle,Indicateur (-1 à 1)
2,CVX,"11,164,955","8,983,959",0.050
0,XOM,"16,784,146","16,181,610",0.015
4,BNP.PA,"2,827,587","2,925,414",-0.008
3,AAPL,"47,422,499","54,223,996",-0.041
1,NVDA,"163,531,152","218,965,467",-0.074


In [ ]:
#@title Score Global de Marché (Direction)
liste_tickers = "AAPL, TSLA, NVDA, XOM, CVX, TTE, BA, AXON, STLA" #@param {type:"string"}

import yfinance as yf
import pandas as pd
import numpy as np

def get_global_score(ticker):
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(period="1y")
        if len(df) < 50: return None

        # 1. SCORE HYPE (Volume Relatif)
        # Mesure si l'action "trend" par rapport à sa normale
        vol_z = (df['Volume'].iloc[-1] - df['Volume'].mean()) / df['Volume'].std()
        hype_score = np.tanh(vol_z / 1.5)

        # 2. SCORE PRESSION (Achat/Vente)
        # Mesure si le flux financier est entrant ou sortant
        price_change = df['Close'].diff().iloc[-1]
        pression_score = np.tanh((price_change / df['Close'].std()) * 5)

        # 3. SCORE TENDANCE (Direction Long Terme)
        # Compare le prix à sa moyenne mobile 50 jours
        ma50 = df['Close'].rolling(window=50).mean().iloc[-1]
        dist_ma50 = (df['Close'].iloc[-1] - ma50) / ma50
        trend_score = np.tanh(dist_ma50 * 10)

        # SCORE FINAL (Moyenne pondérée)
        # On donne un peu plus de poids à la pression et la tendance
        final_score = (hype_score * 0.2) + (pression_score * 0.4) + (trend_score * 0.4)

        return {
            'Ticker': ticker,
            'Buzz (Volume)': round(hype_score, 2),
            'Flux (Achat/Vente)': round(pression_score, 2),
            'Tendance': "HAUSSIÈRE 🟢" if trend_score > 0 else "BAISSIÈRE 🔴",
            'SCORE GLOBAL': round(np.clip(final_score, -1, 1), 3)
        }
    except:
        return None

# --- EXÉCUTION ---
tickers = [t.strip().upper() for t in liste_tickers.split(',')]
results = [res for res in [get_global_score(t) for t in tickers] if res]

# --- AFFICHAGE ---
print("\n" + "="*75)
print(" SYNTHÈSE FINALE : INDICATEUR DE CONVICTION")
print("="*75)
display(pd.DataFrame(results).sort_values(by='SCORE GLOBAL', ascending=False))


 SYNTHÈSE FINALE : INDICATEUR DE CONVICTION


,Ticker,Buzz (Volume),Flux (Achat/Vente),Tendance,SCORE GLOBAL
4,CVX,0.31,0.99,HAUSSIÈRE 🟢,0.638
3,XOM,0.07,1.00,HAUSSIÈRE 🟢,0.589
8,STLA,0.49,0.85,HAUSSIÈRE 🟢,0.497
5,TTE,-0.08,0.88,BAISSIÈRE 🔴,0.278
6,BA,-0.30,-0.15,HAUSSIÈRE 🟢,0.207
7,AXON,-0.19,-0.39,HAUSSIÈRE 🟢,-0.021
1,TSLA,-0.67,0.30,BAISSIÈRE 🔴,-0.092
0,AAPL,-0.19,-0.23,BAISSIÈRE 🔴,-0.318
2,NVDA,-0.37,-0.59,BAISSIÈRE 🔴,-0.346
